In [4]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (1).jpeg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (1).jpg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (1).png  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (10).jpg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (11).jpg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (116).jpg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red disease (117).jpg  
  inflating: Freshwater Fish Disease Aquaculture in south asia/Test/Bacterial Red disease/Bacterial Red dise

In [2]:
!pip install opencv-python

In [11]:
import numpy as np
import cv2
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import os
import joblib as joblib

In [12]:
def get_image_paths(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png']
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            _, ext = os.path.splitext(file)
            if ext.lower() in image_extensions:
                image_paths.append(os.path.join(root, file))
    return image_paths


In [13]:
# good fish 0
image_good = get_image_paths('/content/Freshwater Fish Disease Aquaculture in south asia/Train/Healthy Fish')
labels_good =  np.zeros(len(image_good))


250 250


In [15]:
# bad fish 1
root_path = "/content/Freshwater Fish Disease Aquaculture in south asia/Train/"
file_names = ["Bacterial Red disease",'Bacterial diseases - Aeromoniasis','Bacterial gill disease','Fungal diseases Saprolegniasis','Parasitic diseases','Viral diseases White tail disease']
image_bad = []

for x in file_names:
  image_bad =  np.concatenate((image_bad, get_image_paths(root_path+x)))

labels_bad = np.ones(len(image_bad))

1497 1497


In [17]:
# Feature extraction using HOG
def extract_features(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize image to fixed size
    resized_image = cv2.resize(gray_image, (64, 64))
    # Extract HOG features
    hog_features = hog(resized_image, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2-Hys")
    return hog_features

X = []
for path in image_good:
    image = cv2.imread(path)
    features = extract_features(image)
    X.append(features)

for path in image_bad:
    image = cv2.imread(path)
    features = extract_features(image)
    X.append(features)

# Convert list to numpy array
X = np.array(X)
y = np.concatenate((labels_good,labels_bad))

In [18]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Initialize and train a Support Vector Machine classifier
clf = SVC()
clf.fit(X_train, y_train)

# Predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9085714285714286


In [20]:
# save
joblib.dump(clf, 'fish_classifier_model.pkl')

['fish_classifier_model.pkl']

In [27]:
# Load the model from the file
clf = joblib.load('fish_classifier_model.pkl')
# read image
image = cv2.imread("/content/Freshwater Fish Disease Aquaculture in south asia/Test/Healthy Fish/Healthy Fish (1).jpg")
# Now you can use clf for predictions
y_pred = clf.predict([extract_features(image)])
y_pred

array([0.])